In [45]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from qutip_qip.circuit import QubitCircuit

import pennylane as qml

np.random.seed(42)

In [46]:
dataset = pd.read_csv("./Homework_Datasets/train.csv")
dataset['target'] = dataset['target'].apply(lambda x: 0 if x == 0 else 1)

X=dataset.iloc[: , 1:10].values
y=dataset.iloc[: ,-1].values

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.33, random_state=42)

sc=StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [47]:
n_qubits = 2

def normalize_data(vector):
    norm_vector = np.pi * (2*(vector - vector.min()) / (vector.max() - vector.min()) - 1) / 2
    return norm_vector

def encode_data(vector):
    norm_vector = normalize_data(vector)
    circuit = QubitCircuit(n_qubits)
    circuit.add_gate("H", targets=[0])
    circuit.add_gate("H", targets=[1])
    circuit.add_gate("RX", targets=[0], arg_value=norm_vector[0])
    circuit.add_gate("RX", targets=[1], arg_value=norm_vector[5])
    circuit.add_gate("CNOT", controls=[0], targets=[1])
    circuit.add_gate("RX", targets=[0], arg_value=norm_vector[1])
    circuit.add_gate("RX", targets=[1], arg_value=norm_vector[6])
    circuit.add_gate("CNOT", controls=[1], targets=[0])
    circuit.add_gate("RX", targets=[0], arg_value=norm_vector[2])
    circuit.add_gate("RX", targets=[0], arg_value=norm_vector[3])
    circuit.add_gate("RX", targets=[0], arg_value=norm_vector[7])
    circuit.add_gate("CNOT", controls=[1], targets=[0])
    circuit.add_gate("RX", targets=[0], arg_value=norm_vector[4])
    circuit.add_gate("RX", targets=[0], arg_value=norm_vector[8])
        

    return circuit

In [48]:
dev_kernel = qml.device("lightning.qubit", wires=n_qubits)

projector = np.zeros((2 ** n_qubits, 2 ** n_qubits))
projector[0, 0] = 1

@qml.qnode(dev_kernel)
def kernel(x1, x2):
    """The quantum kernel."""
    encode_data(x1)
    qml.adjoint(encode_data)(x2)
    return qml.expval(qml.Hermitian(projector, wires=range(n_qubits)))

In [49]:
kernel(X_train[0], X_train[0])

1.0

In [50]:
def kernel_matrix(A, B):
    return np.array([[kernel(a, b) for b in B] for a in A])

In [51]:
best_svc = SVC(kernel=kernel_matrix).fit(X_train, y_train)

In [52]:
svc_pred = best_svc.predict(X_test)

# Evaluate the model
print(confusion_matrix(y_test, svc_pred))
print(classification_report(y_test, svc_pred,zero_division=True))

accuracy_score(y_test, svc_pred)

[[  0  55]
 [  0 122]]
              precision    recall  f1-score   support

           0       1.00      0.00      0.00        55
           1       0.69      1.00      0.82       122

    accuracy                           0.69       177
   macro avg       0.84      0.50      0.41       177
weighted avg       0.79      0.69      0.56       177



0.6892655367231638

In [53]:
def circuit_evals_kernel(n_data, split):
    """Compute how many circuit evaluations one needs for kernel-based
       training and prediction."""

    M = int(np.ceil(split * n_data))
    Mpred = n_data - M

    n_training = M * M
    n_prediction = M * Mpred

    return n_training + n_prediction

In [54]:
circuit_evals_kernel(n_data=len(X), split=len(X_train) / (len(X_train) + len(X_test)))

190638